<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/Copy_of_WEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEAT Algorithm
## Test Statistic

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# returns s(w, A, B) for all w in W (passed as argument). Shape: n_words (in W) x 1
def swAB(W, A, B):
  #Calculate cosine-similarity between W and A, W and B
  #print("W: ", W.shape, " A: ", A.shape, " B: ", B.shape)
  WA = cosine_similarity(W,A)
  WB = cosine_similarity(W,B)
  #print('WA shape: ', WA.shape)
  #Take mean along columns
  WAmean = np.mean(WA, axis = 1)
  WBmean = np.mean(WB, axis = 1)
  
  #print('sWAB shape: ', WAmean.shape)
  
  return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
  return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd):
  #Convert the set of words to matrix
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  # Find X U Y
  XuY = list(set(X).union(Y))
  XuYmat = []
  for w in XuY:
    if w.lower() in embd:
      XuYmat.append(embd[w.lower()])
  XuYmat = np.array(XuYmat)
#   print('X U Y Shape: ', XuYmat.shape)
#   print('X Shape: ', Xmat.shape)
#   print('Y Shape: ', Ymat.shape)
#   print('A Shape: ', Amat.shape)
#   print('B Shape: ', Bmat.shape)
  
  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
  
  return d

## P-Value

In [0]:
import random
from itertools import combinations, filterfalse

def random_permutation(iterable, r=None):
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample):
  size_of_permutation = min(len(X), len(Y))
  X_Y = X + Y
  test_stats_over_permutation = []
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
      
  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
    Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
    
  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
  
  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  #print("All: ", test_stats_over_permutation)
  #print("Unpertrubed: ", unperturbed)
  return is_over.sum() / is_over.size

## Test on sample input

In [0]:
X = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"] #Instruments
Y = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"] #Weapons
A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"] #Pleasant
B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"] #Unpleasant

#Load word embeddings
#load gensim formatted Full Glove embeddings
#!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''


glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

print('WEAT d = ', weat_effect_size(X, Y, A, B, glove))
print('WEAT p = ', weat_p_value(X, Y, A, B, glove, 1000))

The glove embedding has been loaded!
WEAT d =  1.5495627
Unpertrubed:  2.2905553244054317
WEAT p =  0.0


## WEAT with conceptor debiased embeddings

### Compute the conceptor matrix for all words and gender specific words.

In [0]:
#Compute the conceptor matrix
def process_cn_matrix(subspace, alpha = 2):
  #print("starting...")
  #x = orig_embd.vectors
  #print(subspace.shape)
  
  #Calculate the correlation matrix
  R = subspace.dot(subspace.T)/(subspace.shape[1])
  #print("R calculated")
  
  #Calculate the conceptor matrix
  C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(subspace.shape[0])))
  #print("C calculated")
  
  #Calculate the negation of the conceptor matrix
  negC = np.eye(subspace.shape[0]) - C
  #print("negC calculated")
  
  return negC

def apply_conceptor(x, C):
  #Post-process the vocab matrix
  newX = (C @ x).T
  print(newX.shape)
  return newX

Load embeddings of all words from the ref. wordlist from a specific embedding

In [0]:
#Arguments - embd: The word embeddings in the form of a dict || wikiWordsPath: List of words to be considered
def load_all_vectors(embd, wikiWordsPath):
  all_words_index = {}
  all_words_mat = []
  with open(wikiWordsPath, "r+") as f_in:
    ind = 0
    for line in f_in:
      word = line.split(' ')[0]
      if word in embd:
        all_words_index[word] = ind
        all_words_mat.append(embd[word])
        ind = ind+1
        
  return all_words_index, all_words_mat

def load_subspace_vectors(embd, subspace_words):
  subspace_embd_mat = []
  ind = 0
  for word in subspace_words:
    if word in embd:
      subspace_embd_mat.append(embd[word])
      ind = ind+1
      
  return subspace_embd_mat

In [66]:
all_words_index, all_words_mat = load_all_vectors(glove, wikiWordsPath)

aaa = list(all_words_index.keys())
X = WEATLists.W_5_Unused_full_list_European_American_names
print(X)
a = ['T' for w in X if w.lower() in aaa]
print(a)
print(np.array(a).shape)

['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil', 'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith', 'Sarah']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
(18,)


Conceptor all words and store it in a dictonary

### Load all word lists - Subspace

In [10]:
#If running on local
!mkdir content
%cd content
# General word list
!wget https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20150602-words-frequency.txt
!git clone https://github.com/PrincetonML/SIF
    
# Gender word lists
!git clone https://github.com/uclanlp/gn_glove
!git clone https://github.com/uclanlp/corefBias
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
    
# our code for debiasing -- also includes word lists    
!git clone https://github.com/jsedoc/ConceptorDebias

mkdir: cannot create directory ‘content’: File exists
/home/saketk/ConceptorDebias/content
--2019-02-28 22:28:27--  https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20150602-words-frequency.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23741395 (23M) [text/plain]
Saving to: ‘enwiki-20150602-words-frequency.txt’

enwiki-20150602-wor 100%[===================>]  22.64M  29.6MB/s    in 0.8s    

2019-02-28 22:28:28 (29.6 MB/s) - ‘enwiki-20150602-words-frequency.txt’ saved [23741395/23741395]

fatal: destination path 'SIF' already exists and is not an empty directory.
fatal: destination path 'gn_glove' already exists and is not an empty directory.
fatal: destination path 'corefBias' already exists and is not an empty d

In [16]:
!ls

CN.ipynb				     flair_bert.ipynb
ConceptorDebias				     flair_elmo_bert.ipynb
conceptor_fxns.py			     gn_glove
ConceptorPostPro_DEMO.ipynb		     Greenwald_1998_Perez_2010_lists.py
content					     LICENSE
Copy_of_WEAT.ipynb			     load_word_lists.py
corefBias				     male.txt
Debias_BERT.ipynb			     plot_pc.py
Debias_Contextualized_Representations.ipynb  __pycache__
Debiasing_WE.ipynb			     README.md
Debias_paper_section_1.ipynb		     Re_implement_CN.ipynb
EEC_with_CN.ipynb			     SIF
enwiki-20150602-words-frequency.txt	     WEAT.ipynb
female.txt				     WEAT_lists.py


In [0]:
from load_word_lists import *

gender_list_pronouns = WEATLists.W_7_Male_terms + WEATLists.W_7_Female_terms + WEATLists.W_8_Male_terms + WEATLists.W_8_Female_terms
gender_list_pronouns = list(set(gender_list_pronouns))

gender_list_extended = male_vino_extra + female_vino_extra + male_gnGlove + female_gnGlove
gender_list_extended = list(set(gender_list_extended))

gender_list_propernouns = male_cmu + female_cmu
gender_list_propernouns = list(set(gender_list_propernouns))

gender_list_all = gender_list_pronouns + gender_list_extended + gender_list_propernouns
gender_list_all = list(set(gender_list_all))

race_list = WEATLists.W_3_Unused_full_list_European_American_names + WEATLists.W_3_European_American_names + WEATLists.W_3_Unused_full_list_African_American_names + WEATLists.W_3_African_American_names + WEATLists.W_4_Unused_full_list_European_American_names + WEATLists.W_4_European_American_names + WEATLists.W_4_Unused_full_list_African_American_names + WEATLists.W_4_African_American_names + WEATLists.W_5_Unused_full_list_European_American_names + WEATLists.W_5_European_American_names + WEATLists.W_5_Unused_full_list_African_American_names + WEATLists.W_5_African_American_names 
race_list = list(set(race_list))

### Load different embeddings

Glove

In [29]:
#Load word embeddings
#download gensim formatted Full Glove embeddings
#!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

The glove embedding has been loaded!


In [71]:
X = WEATLists.W_5_Unused_full_list_European_American_names
print(X)
a = [glove[w] for w in X if w.lower() in glove]
print(np.array(a).shape)
glove['Brad']
#glove['brad']

['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil', 'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith', 'Sarah']
(18, 300)


array([-2.5609e-01, -1.5396e-01, -5.0346e-01,  2.5138e-01,  3.3952e-01,
        2.6105e-01,  3.0257e-01, -1.1273e-01,  4.6391e-02,  2.1197e+00,
        1.9856e-01, -3.1840e-02, -4.4469e-01, -8.6130e-03, -3.6665e-01,
       -7.8715e-02, -1.8301e-01, -3.5935e-01, -2.9033e-01,  3.2755e-01,
       -1.1633e-01, -5.3637e-02, -3.1589e-01, -9.0069e-02,  5.4889e-02,
        1.0793e-01, -7.6549e-02,  3.2448e-01, -3.7830e-01, -7.1353e-03,
       -9.2837e-02, -3.4702e-01, -4.4372e-02, -3.6582e-02,  6.6447e-02,
        4.3170e-01,  1.4958e-01,  4.1407e-01, -7.3515e-04, -5.4028e-01,
        3.4552e-01, -5.6781e-02, -1.9483e-01,  3.7953e-01,  4.6593e-02,
        4.1634e-02, -5.3288e-01,  1.4703e-01,  2.6140e-02, -1.6343e-03,
        3.7304e-01, -1.7348e-01,  2.5698e-02, -2.5498e-01, -1.9006e-01,
       -5.8404e-01, -6.0172e-03,  4.6960e-02,  1.4949e-02,  4.7884e-01,
        1.8709e-01,  1.1012e-01, -2.9987e-01,  3.7356e-02, -4.1855e-02,
        1.2758e-02,  1.5549e-01, -7.8430e-02,  3.5795e-01, -2.11

Word2vec

In [30]:
#load gensim formatted Full Word2vec embeddings
#!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
#!gunzip GoogleNews-vectors-negative300.bin.gz
  
import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

word2vec = KeyedVectors.load_word2vec_format(resourceFile + 'GoogleNews-vectors-negative300.bin', binary=True)
print('The word2vec embedding has been loaded!')

The word2vec embedding has been loaded!


Fasttext

In [32]:
#!gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh

import gensim
from gensim.models.keyedvectors import KeyedVectors  

resourceFile = ''
fasttext = KeyedVectors.load_word2vec_format(resourceFile + 'fasttext.bin', binary=True)
print('The fasttext embedding has been loaded!')

The fasttext embedding has been loaded!


Elmo

In [0]:
#!gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
  
import pickle
data = pickle.load(open("elmo_embeddings_emma_brown.pkl", "rb"))

def pick_embeddings(corpus, sent_embs, word_list):
    X = []
    labels = {}
    sents = []
    ind = 0
    for i, s in enumerate(corpus):
        for j, w in enumerate(s):
            if w.lower() in word_list:
                X.append(sent_embs[i][j])
                if w.lower() in labels:
                  labels[w.lower()].append(ind)
                else:
                  labels[w.lower()] = [ind]
                sents.append(s)
                ind = ind + 1
    return (X, labels, sents)
  
def get_word_list(path):
    word_list = []
    with open(path, "r+") as f_in:
      for line in f_in:
        word = line.split(' ')[0]
        word_list.append(word.lower())

    return word_list

def load_subspace_vectors_contextual(all_mat, all_index, subspace_list):
    subspace_mat = []
    for w in subspace_list:
      if w.lower() in all_index:
        for i in all_index[w.lower()]:
          #print(type(i))
          subspace_mat.append(all_mat[i])
    #subspace_mat = [all_mat[i,:] for i in all_index[w.lower()] for w in subspace_list if w.lower() in all_index]
    print("Subspace: ", np.array(subspace_mat).shape)
    return subspace_mat

In [0]:
import nltk
from nltk.corpus import brown

#nltk.download('brown')

brown_corpus = brown.sents()
elmo = data['brown_embs']


In [111]:
print(np.array(elmo_brown_mat).shape)
print(len(list(elmo_brown_index.keys())))
print(len(wiki_words))

(972811, 768)
34391
188033


In [113]:
a = [[1,2,3], [4,5,6]]
np.mean(a, axis = 0)

array([2.5, 3.5, 4.5])

Bert

In [95]:
!pip install nltk

    100% |████████████████████████████████| 1.4MB 15.2MB/s 
  Running setup.py install for nltk ... - \ | / - \ | done


###Post-process (CN) all embeddings using a particular subspace

In [147]:
import numpy as np
from ConceptorDebias.conceptor_fxns import AND
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['elmo']
all_subspace = ['without_conceptor','gender_list_pronouns', 'gender_list_extended','gender_list_propernouns', 'gender_list_all', 'gender_list_and']
# all_subspace = ['without_conceptor', 'race_list']

math = WEATLists.W_7_Math
arts = WEATLists.W_7_Arts
male = WEATLists.W_7_Male_terms
female = WEATLists.W_7_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    wiki_words = get_word_list('SIF/auxiliary_data/enwiki_vocab_min200.txt')
    #all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd, wiki_words)
    print("All mat: ", np.array(all_words_mat).shape)
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_conceptor' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_conceptor':
      #CN all word embeddings using the respective subspace
      if subspace == 'gender_list_and':
        if embd == 'elmo' or embd == 'bert':
          subspace_words_mat1 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_pronouns)
          cn1 = process_cn_matrix(np.array(subspace_words_mat1).T)

          subspace_words_mat2 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_extended)
          cn2 = process_cn_matrix(np.array(subspace_words_mat2).T)

          subspace_words_mat3 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_propernouns)
          cn3 = process_cn_matrix(np.array(subspace_words_mat3).T)

          cn = AND(cn1, AND(cn2, cn3))
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
          print("All mat CN: ", np.array(all_words_cn).shape)
        else:
          subspace_words_mat1 = load_subspace_vectors(curr_embd, gender_list_pronouns)
          cn1 = process_cn_matrix(np.array(subspace_words_mat1).T)

          subspace_words_mat2 = load_subspace_vectors(curr_embd, gender_list_extended)
          cn2 = process_cn_matrix(np.array(subspace_words_mat2).T)

          subspace_words_mat3 = load_subspace_vectors(curr_embd, gender_list_propernouns)
          cn3 = process_cn_matrix(np.array(subspace_words_mat3).T)

          cn = AND(cn1, AND(cn2, cn3))
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      else:
        #Load all embeddings of the subspace as a matrix
        if embd == 'elmo' or embd == 'bert':
          subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
          cn = process_cn_matrix(np.array(subspace_words_mat).T)
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
          print("Subspace mat: ", np.array(subspace_words_mat).shape)
        else:
          subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
          cn = process_cn_matrix(np.array(subspace_words_mat).T)
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    print("LAST: ", np.array(all_words["a"]).shape)
    if subspace == 'without_conceptor':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("Without conceptor")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("With conceptor: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

All mat:  (972811, 768)
All CN:  (972811, 768)
LAST:  (768,)
Without conceptor
WEAT d =  0.9415434
WEAT p =  0.015
Subspace:  (27902, 768)
(972811, 768)
Subspace mat:  (27902, 768)
All CN:  (972811, 768)
LAST:  (768,)
With conceptor:  elmo gender_list_pronouns
WEAT d =  0.34302947657573135
WEAT p =  0.174
Subspace:  (33163, 768)
(972811, 768)
Subspace mat:  (33163, 768)
All CN:  (972811, 768)
LAST:  (768,)
With conceptor:  elmo gender_list_extended
WEAT d =  0.3413683848252179
WEAT p =  0.219
Subspace:  (27392, 768)
(972811, 768)
Subspace mat:  (27392, 768)
All CN:  (972811, 768)
LAST:  (768,)
With conceptor:  elmo gender_list_propernouns
WEAT d =  1.1114971447774085
WEAT p =  0.025
Subspace:  (60555, 768)
(972811, 768)
Subspace mat:  (60555, 768)
All CN:  (972811, 768)
LAST:  (768,)
With conceptor:  elmo gender_list_all
WEAT d =  1.4051261712409355
WEAT p =  0.0
Subspace:  (27902, 768)
Subspace:  (33163, 768)
Subspace:  (27392, 768)
(972811, 768)
All mat CN:  (972811, 768)
All CN:  (9

,Embedding,Subspace,Effect Size,p-value
0,elmo,without_conceptor,0.9415434,0.015
1,elmo,gender_list_pronouns,0.34302947657573135,0.174
2,elmo,gender_list_extended,0.3413683848252179,0.219
3,elmo,gender_list_propernouns,1.1114971447774085,0.025
4,elmo,gender_list_all,1.4051261712409355,0.0
5,elmo,gender_list_and,1.3059196442097278,0.008


In [127]:
if w.lower() not in all_words_index:
  print("T")
#load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_pronouns)

T


### Calculate WEAT scores

In [0]:
science = WEATLists.W_8_Science
arts = WEATLists.W_8_Arts
male = WEATLists.W_8_Male_terms
female = WEATLists.W_8_Female_terms

#WITHOUT CONCEPTOR
print("Without conceptor")
print('WEAT d = ', weat_effect_size(science, arts, male, female, curr_embd))
print('WEAT p = ', weat_p_value(science, arts, male, female, curr_embd, 1000))

#WITH CONCEPTOR
print("With conceptor")
print('WEAT d = ', weat_effect_size(science, arts, male, female, all_words))
print('WEAT p = ', weat_p_value(science, arts, male, female, all_words, 1000))

Without conceptor
WEAT d =  1.2780356
Unpertrubed:  0.3456038013100624
WEAT p =  0.003
With conceptor
WEAT d =  0.681611936758009
Unpertrubed:  0.10700200023922478
WEAT p =  0.103


## WEAT algorithm from GITHUB gist
REF: https://gist.github.com/SandyRogers/e5c2e938502a75dcae25216e4fae2da5

In [0]:
class WEATTest(object):
    """
    Perform WEAT (Word Embedding Association Test) bias tests on a language model.
    Follows from Caliskan et al 2017 (10.1126/science.aal4230).
    """
    
    instruments = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"]
    weapons = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"]
    flowers = ["aster", "clover", "hyacinth", "marigold", "poppy", "azalea", "crocus", "iris", "orchid", "rose", "blue-bell", "daffodil", "lilac", "pansy", "tulip", "buttercup", "daisy", "lily", "peony", "violet", "carnation", "gladiola", "magnolia", "petunia", "zinnia"]
    insects = ["ant", "caterpillar", "flea", "locust", "spider", "bedbug", "centipede", "fly", "maggot", "tarantula", "bee", "cockroach", "gnat", "mosquito", "termite", "beetle", "cricket", "hornet", "moth", "wasp", "blackfly",
    "dragonfly", "horsefly", "roach", "weevil"]
    pleasant = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"]
    unpleasant = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"]
    math = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
    arts = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
    male = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
    female = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
    science = ["science", "technology", "physics", "chemistry", "Einstein", "NASA", "experiment", "astronomy"]
    arts_2 = ["poetry", "art", "Shakespeare", "dance", "literature", "novel", "symphony", "drama"]
    male_2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
    female_2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "he"]
    mental_disease = ["sad", "hopeless", "gloomy", "tearful", "miserable", "depressed"]
    physical_disease = ["sick", "illness", "influenza", "disease", "virus", "cancer"]
    temporary = ["impermanent", "unstable", "variable", "fleeting", "short-term", "brief", "occasional"]
    permanent = ["stable", "always", "constant", "persistent", "chronic", "prolonged", "forever"]
    
    def __init__(self, model):
        """Setup a Word Embedding Association Test for a given spaCy language model.
        
        EXAMPLE:
            >>> nlp = spacy.load('en_core_web_md')
            >>> test = WEATTest(nlp)
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
        """
        self.model = model

    @staticmethod
    def word_association_with_attribute(self, w, A, B):
        return np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(a).reshape(1,-1)) for a in A]) - np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(b).reshape(1,-1)) for b in B])

    @staticmethod
    def differential_assoication(self, X, Y, A, B):
        return np.sum([self.word_association_with_attribute(self, x, A, B) for x in X]) - np.sum([self.word_association_with_attribute(self, y, A, B) for y in Y])

    @staticmethod
    def weat_effect_size(self, X, Y, A, B):
        return (
            np.mean([self.word_association_with_attribute(self, x, A, B) for x in X]) -
            np.mean([self.word_association_with_attribute(self, y, A, B) for y in Y])
        ) / np.std([self.word_association_with_attribute(self, w, A, B) for w in X + Y])

    @staticmethod
    def random_permutation(self, iterable, r=None):
        pool = tuple(iterable)
        r = len(pool) if r is None else r
        return tuple(random.sample(pool, r))

    @staticmethod
    def weat_p_value(self, X, Y, A, B, sample):
        size_of_permutation = min(len(X), len(Y))
        X_Y = X + Y
        observed_test_stats_over_permutations = []

        if not sample:
            permutations = combinations(X_Y, size_of_permutation)
        else:
            permutations = [self.random_permutation(self, X_Y, size_of_permutation) for s in range(sample)]
        print(np.array(X_Y).shape)
        for Xi in permutations:
            Yi = filterfalse(lambda w:w in Xi, X_Y)
            observed_test_stats_over_permutations.append(self.differential_assoication(self, Xi, Yi, A, B))

        unperturbed = self.differential_assoication(self, X, Y, A, B)
        is_over = np.array([o > unperturbed for o in observed_test_stats_over_permutations])
        return is_over.sum() / is_over.size

    @staticmethod
    def weat_stats(X, Y, A, B, self, sample_p=None):
        test_statistic = self.differential_assoication(self, X, Y, A, B)
        effect_size = self.weat_effect_size(self, X, Y, A, B)
        p = self.weat_p_value(self, X, Y, A, B, sample=sample_p)
        return test_statistic, effect_size, p

    def run_test(self, target_1, target_2, attributes_1, attributes_2, sample_p=None):
        """Run the WEAT test for differential association between two 
        sets of target words and two seats of attributes.
        
        EXAMPLE:
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
            >>> test.run_test(a, b, c, d, sample_p=1000) # use 1000 permutations for p-value calculation
            >>> test.run_test(a, b, c, d, sample_p=None) # use all possible permutations for p-value calculation
            
        RETURNS:
            (d, e, p). A tuple of floats, where d is the WEAT Test statistic, 
            e is the effect size, and p is the one-sided p-value measuring the
            (un)likeliness of the null hypothesis (which is that there is no
            difference in association between the two target word sets and
            the attributes).
            
            If e is large and p small, then differences in the model between 
            the attribute word sets match differences between the targets.
        """
        X = [list(self.model[w]) for w in target_1]
        Y = [list(self.model[w]) for w in target_2]
        A = [list(self.model[w]) for w in attributes_1]
        B = [list(self.model[w]) for w in attributes_2]
        print(X)
        return self.weat_stats(X, Y, A, B, self, sample_p)


## Code test

In [0]:
#nlp = spacy.load('glove')
test = WEATTest(glove)
test.run_test(WEATTest.instruments, WEATTest.weapons, WEATTest.pleasant, WEATTest.unpleasant, 1000)

The glove embedding has been loaded!
X Shape:  (3, 300)
Y Shape:  (3, 300)
A Shape:  (2, 300)
B Shape:  (2, 300)
X U Y Shape:  (6, 300)
WA shape:  (3, 2)
sWAB shape:  (3,)
WA shape:  (3, 2)
sWAB shape:  (3,)
WA shape:  (6, 2)
sWAB shape:  (6,)
WEAT d =  1.8613524


In [0]:
all_subspace = ['gender_list_pronouns','gender_list_extended','gender_list_propernouns']
op = []
for s in all_subspace:
  aa = s
  print(aa)
  a = 10.98
  b = 99.09
  temp = [s, a, b]
  op.append(temp)
print(op)

gender_list_pronouns
gender_list_extended
gender_list_propernouns
[['gender_list_pronouns', 10.98, 99.09], ['gender_list_extended', 10.98, 99.09], ['gender_list_propernouns', 10.98, 99.09]]
